In [2]:
import numpy as np
import pandas as pd
import plotly.express as px
from datetime import datetime, timedelta
from StreamlitApp_S1.filtros_visualizaciones import *
import json
import folium
import branca.colormap as cm

In [79]:
df_demanda = pd.read_csv('./data/processed/DF_DEMANDA_10_25_LIMPIO.csv')
df_demanda = df_demanda[df_demanda['titulo'] == 'Demanda']
df_generacion = pd.read_csv('./data/processed/DF_GENERACION_10_25_LIMPIO.csv')
df_intercambios = pd.read_csv('./data/processed/DF_INTERCAMBIOS_10_25_LIMPIO.csv')
df_balance = pd.read_csv('./data/processed/DF_BALANCE_10_25_LIMPIO.csv')

In [141]:
df1 = df_demanda
df2 = df_generacion

In [142]:
df1 = df1[(df1['año'] == 2022) | (df1['año'] == 2024)].reset_index(drop=True)
df2 = df2[(df2['año'] == 2022) | (df2['año'] == 2024)].reset_index(drop=True)

In [143]:
df1 = df1[df1['zona'] == 'nacional']
df2 = df2[df2['zona'] == 'nacional']

In [144]:
df1 = df1.groupby('fecha').sum('valor_(MWh)').reset_index(drop=False)
df2 = df2.groupby('fecha').sum('valor_(MWh)').reset_index(drop=False)

In [145]:
df = pd.merge(df1, df2, on='fecha', how='left')


In [146]:
df['valor_(GWh)'] = (df['valor_(MWh)_y'] - df['valor_(MWh)_x']) * 0.001
df.head()

,fecha,composite_x,valor_(MWh)_x,porcentaje_x,año_x,mes_x,dia_x,composite_y,valor_(MWh)_y,porcentaje_y,año_y,mes_y,dia_y,valor_(GWh)
0,2022-01-01,0,535144.786,1.0,2022,1,1,0,550421.606,1.0,30330,15,15,15.276820
1,2022-01-02,0,578855.186,1.0,2022,1,2,0,572434.505,1.0,30330,15,30,-6.420681
2,2022-01-03,0,691970.774,1.0,2022,1,3,0,660743.562,1.0,30330,15,45,-31.227212
3,2022-01-04,0,718047.874,1.0,2022,1,4,0,737194.073,1.0,30330,15,60,19.146199
4,2022-01-05,0,715070.234,1.0,2022,1,5,0,752635.612,1.0,30330,15,75,37.565378


In [ ]:
df = df[['fecha', 'año_x', 'mes_x', 'dia_x', 'valor_(GWh)']].rename(columns={'año_x': 'año', 'mes_x': 'mes', 'dia_x': 'dia'})
df.head()

,fecha,año,mes,dia,valor_(GWh)
0,2022-01-01,2022,1,1,15.276820
1,2022-01-02,2022,1,2,-6.420681
2,2022-01-03,2022,1,3,-31.227212
3,2022-01-04,2022,1,4,19.146199
4,2022-01-05,2022,1,5,37.565378


In [ ]:
df_estadisticas = df.groupby('año')['valor_(GWh)'].agg(['mean', 'median', 'max', 'min'])


,mean,median,max,min
año,,,,
2022,60.605134,65.888768,149.554544,-47.086554
2024,36.711234,34.858454,160.530708,-65.245785


### Demanda para 7, 30, 365 días y el histórico


In [49]:
today = datetime.now()
delta = timedelta(days=365)
start_date = (today - delta).strftime("%Y-%m-%dT%H:%M")
end_date = today.strftime("%Y-%m-%d")

df_demanda_filtered_1 = df_demanda[(df_demanda['año'] == 2022) & (df_demanda['año'] == 2024)].reset_index(drop=True)
df_demanda_filtered_1 = df_demanda_filtered_1[df_demanda_filtered_1['zona'] == 'nacional']
df_demanda_filtered_1 = df_demanda_filtered_1.groupby(['fecha', 'año', 'mes' ,'dia'])['valor_(MWh)'].sum().reset_index()

df_demanda_filtered_2 = df_balance[(start_date <= df_balance['fecha']) & (df_balance['fecha']<= end_date)].reset_index(drop=True)
df_demanda_filtered_2 = df_demanda_filtered_2[df_demanda_filtered_2['zona'] == 'nacional']
df_demanda_filtered_2 = df_demanda_filtered_2.groupby(['fecha', 'año', 'mes' ,'dia']).sum('valor_(MWh)').reset_index(drop=False)

df_demanda_filtered_2

,fecha,año,mes,dia,composite,valor_(MWh),porcentaje
0,2024-02-24,2024,2,24,0,657800.038,3.088684
1,2024-02-25,2024,2,25,0,626143.633,3.059938
2,2024-02-26,2024,2,26,0,730573.860,3.052669
3,2024-02-27,2024,2,27,0,746142.904,3.046888
4,2024-02-28,2024,2,28,0,732016.128,3.072538
...,...,...,...,...,...,...,...
358,2025-02-16,2025,2,16,0,599087.640,3.083977
359,2025-02-17,2025,2,17,0,703417.661,3.055465
360,2025-02-18,2025,2,18,0,729142.540,3.024132
361,2025-02-19,2025,2,19,0,719164.014,3.005061


In [32]:
df_demanda_filtered_1.head()

,titulo,composite,fecha_extraccion,categoria,zona,valor_(MWh),porcentaje,fecha,año,mes,dia,dia_semana,ID
0,Demanda,0,2025-02-20,Evolución de la demanda,nacional,657800.038,1.0,2024-02-24,2024,2,24,sábado,20240224-a72f5091b30b451d8a1863d4f6cbc558
1,Demanda,0,2025-02-20,Evolución de la demanda,nacional,626143.633,1.0,2024-02-25,2024,2,25,domingo,20240225-22927b4fb007431e9282aa8e1619c5a7
2,Demanda,0,2025-02-20,Evolución de la demanda,nacional,730573.860,1.0,2024-02-26,2024,2,26,lunes,20240226-1785d16db2d741a7a34115fb14e9c835
3,Demanda,0,2025-02-20,Evolución de la demanda,nacional,746142.904,1.0,2024-02-27,2024,2,27,martes,20240227-bf9dae4a61064ad2836aaea5836c7014
4,Demanda,0,2025-02-20,Evolución de la demanda,nacional,732016.128,1.0,2024-02-28,2024,2,28,miércoles,20240228-96f2fe34920944d8a9e2afb572282c15


In [33]:
px.line(data_frame=df_demanda_filtered_1,
        x='fecha',
        y='valor_(MWh)',
        color='zona'
        )

In [34]:
df_demanda_filtered_2.head()

,fecha,composite,valor_(MWh),porcentaje,año,mes,dia
0,2024-02-24,0,1315600.076,15.443496,125488,124,1488
1,2024-02-25,0,1252287.266,14.387174,123464,122,1525
2,2024-02-26,0,1461147.720,14.424961,123464,122,1586
3,2024-02-27,0,1492285.808,14.396581,123464,122,1647
4,2024-02-28,0,1464032.256,14.458491,123464,122,1708


In [35]:
px.line(data_frame=df_demanda_filtered_2,
        x='fecha',
        y='valor_(MWh)',
        )

### Intercambios

In [11]:
df_intercambios.head()


,titulo,tipo,composite,fecha_extraccion,categoria,frontera,zona,valor_(MWh),porcentaje,fecha,año,mes,dia,dia_semana,ID
0,Exportación,export,0,2025-02-20,Intercambios por frontera con Francia,Francia,nacional,-3344.187,0.227339,2011-01-01,2011,1,1,sábado,20110101-87c3c348e5b545558cab38d5f0b58a52
1,Exportación,export,0,2025-02-20,Intercambios por frontera con Francia,Francia,nacional,-7900.880,0.559291,2011-01-02,2011,1,2,domingo,20110102-0e106c9478e24ae9ac2abd76fffb606b
2,Exportación,export,0,2025-02-20,Intercambios por frontera con Francia,Francia,nacional,-13808.850,0.924363,2011-01-03,2011,1,3,lunes,20110103-2695d0c7197b4a67b3500b78254420c6
3,Exportación,export,0,2025-02-20,Intercambios por frontera con Francia,Francia,nacional,-15848.206,0.990490,2011-01-04,2011,1,4,martes,20110104-97616bcced384ca6934bbdcb5c68c9f8
4,Exportación,export,0,2025-02-20,Intercambios por frontera con Francia,Francia,nacional,-19388.927,0.858016,2011-01-05,2011,1,5,miércoles,20110105-86ccd402f05c410683e766641181b857


In [3]:
df_intercambios['frontera'].unique()

array(['Francia', 'Portugal', 'Marruecos', 'Andorra',
       'Enlace Península-Baleares'], dtype=object)

In [4]:
today = datetime.strptime(df_intercambios['fecha'].max(), "%Y-%m-%d")
print(today)
delta = timedelta(days=7)
print(delta)
start_date = (today - delta).strftime("%Y-%m-%d")
end_date = today.strftime("%Y-%m-%d")

print(start_date)
print(end_date)

2025-02-19 00:00:00
7 days, 0:00:00
2025-02-12
2025-02-19


In [47]:
df_intercambios_filtrado_1 = p7_30_365_hist(df_intercambios, 30)[0]
df_intercambios_filtrado_2 = df_intercambios_filtrado_1.groupby(['frontera','tipo'])['valor'].sum().reset_index()
df_intercambios_filtrado_2 = df_intercambios_filtrado_2[df_intercambios_filtrado_2['frontera'] != 'Enlace Península-Baleares']
df_intercambios_filtrado_2

,frontera,tipo,valor
0,Andorra,export,-11468.720
1,Andorra,import,0.000
4,Francia,export,-480818.161
5,Francia,import,234806.459
6,Marruecos,export,-62437.392
7,Marruecos,import,11325.528
8,Portugal,export,-206799.985
9,Portugal,import,324271.830


In [48]:
exportacion_total = df_intercambios_filtrado_2.groupby('tipo').sum()['valor'].values[0]
importacion_total = df_intercambios_filtrado_2.groupby('tipo').sum()['valor'].values[1]

intercambios_nac = {
    'frontera': ['Intercambios', 'Intercambios'],
    'tipo': ['export', 'import'],
    'valor': [exportacion_total, importacion_total]
}

intercambios_nac = pd.DataFrame(intercambios_nac)

In [49]:
df_intercambios_filtrado_2 = pd.concat([df_intercambios_filtrado_2, intercambios_nac])

In [50]:
df_saldo = df_intercambios_filtrado_2.groupby('frontera').sum().reset_index()
df_saldo['tipo'] = df_saldo['tipo'].apply(lambda x: 'saldo')
df_saldo = pd.concat([df_intercambios_filtrado_2, df_saldo]).reset_index(drop=True)
df_saldo

,frontera,tipo,valor
0,Andorra,export,-11468.720
1,Andorra,import,0.000
2,Francia,export,-480818.161
3,Francia,import,234806.459
4,Marruecos,export,-62437.392
5,Marruecos,import,11325.528
6,Portugal,export,-206799.985
7,Portugal,import,324271.830
8,Intercambios,export,-761524.258
9,Intercambios,import,570403.817


In [51]:
coord_paises = {
                    "frontera": ["Intercambios", "Portugal", "Francia", "Andorra", "Marruecos"],
                    "latitud": [40.4168, 38.7169, 45.83362, 42.5078, 33.2514],
                    "longitud": [-3.7038, -9.1399, 1.24759, 1.5211, -5.372]
                }

df_coord = pd.DataFrame(coord_paises)
# df_intercambio_red = df_intercambios[['tipo', 'frontera', 'zona', 'valor', 'porcentaje', 'datetime']]
# df_intercambio_red = df_intercambio_red[df_intercambio_red['zona'] == 'nacional']
# df_intercambio_red['datetime'] = pd.to_datetime(df_intercambio_red['datetime'], utc=True)
# df_intercambio_total = df_intercambio_red.groupby(['frontera','type'])['value'].sum().reset_index()#.sort_values(by='value', ascending=False).reset_index(drop=True)
# df_intercambios_tot_coord = df_intercambio_total[df_intercambio_total['type'] == 'saldo'].reset_index(drop=True)
# df_intercambios_tot_coord = df_intercambios_tot_coord.merge(df_coord, on='frontera', how='left')
# df_intercambios_tot_coord

In [52]:
df_saldo = df_saldo.merge(df_coord, on='frontera', how='left')
# df_saldo['color'] = df_saldo['valor'].apply(lambda x: 'green' if x < 0 else 'red')

In [53]:
df_saldo

,frontera,tipo,valor,latitud,longitud
0,Andorra,export,-11468.720,42.50780,1.52110
1,Andorra,import,0.000,42.50780,1.52110
2,Francia,export,-480818.161,45.83362,1.24759
3,Francia,import,234806.459,45.83362,1.24759
4,Marruecos,export,-62437.392,33.25140,-5.37200
5,Marruecos,import,11325.528,33.25140,-5.37200
6,Portugal,export,-206799.985,38.71690,-9.13990
7,Portugal,import,324271.830,38.71690,-9.13990
8,Intercambios,export,-761524.258,40.41680,-3.70380
9,Intercambios,import,570403.817,40.41680,-3.70380


In [71]:
spain_map = folium.Map(location=[40.4637, -3.7492], tiles="openstreetmap", zoom_start=5, min_zoom=5, max_zoom=7, max_bounds=True)

geo_json_path = "./StreamlitApp_S1/world_countries.json"
with open(geo_json_path, "r", encoding="utf-8") as file:
    geo_json_data = json.load(file)

colormap = cm.LinearColormap(
colors=['red', 'gray', 'green'],  # Rojo para importaciones > exportaciones, verde para exportaciones > importaciones
vmin=df_saldo['valor'].min(),
vmax=df_saldo['valor'].max()
)

for pais, lat, lon in zip(df_coord['frontera'], df_coord['latitud'], df_coord['longitud']):

    exp = df_saldo[(df_saldo['tipo'] == 'export') & (df_saldo['frontera'] == pais)]['valor']* 0.001
    imp = df_saldo[(df_saldo['tipo'] == 'import') & (df_saldo['frontera'] == pais)]['valor'] * 0.001
    saldo = df_saldo[(df_saldo['tipo'] == 'saldo') & (df_saldo['frontera'] == pais)]['valor'] * 0.001

    popup_html = f"""
        <div style="width: 250px; font-size: 14px;">
        <b>📍{pais}</b><br>
        📤 Exportado: {round(exp, 1).values[0]} GWh<br>
        📥 Importado: {round(imp, 1).values[0]} GWh<br>
        ⚖️ Saldo: {round(saldo, 1).values[0]} GWh
        </div>
        """
    popup = folium.Popup(popup_html, max_width=300)
    if saldo.values[0] < 0:
        color = 'green'
    else:
        color = 'blue'

    folium.Marker(
        location=[lat, lon],
        popup=popup,
        icon=folium.Icon(color=color, icon="bolt", prefix="fa")
        ).add_to(spain_map)
    
folium.Choropleth(
    geo_data=geo_json_data,
    data=df_saldo[df_saldo['tipo'] == 'saldo'],
    columns=["frontera", "valor"],
    nan_fill_color="gray",
    nan_fill_opacity=0.4,
    key_on="feature.properties.ADMIN",
    fill_color="PuBu",  # Escala de color de folium
    fill_opacity=0.7,
    line_opacity=0.2,
).add_to(spain_map)

colormap.add_to(spain_map)
spain_map

In [43]:
spain_map = folium.Map(location = [40.4637, -3.7492], zoom_start = 5, min_zoom = 5)

coord = folium.map.FeatureGroup()

geo_json = "/Data/world_countries.json"

for pais, lat, lon in zip(df_coord["frontera"], df_coord["latitud"], df_coord["longitud"]):

    row = df_intercambio_total.loc[df_intercambio_total['frontera'] == pais].reset_index(drop=True)
    # print(row)
    exp = row[row['tipo'] == 'export']['valor'].values[0]
    # print(exp)
    imp = row[row['tipo'] == 'import']['valor'].values[0]
    # print(imp)
    saldo = row[row['tipo'] == 'saldo']['valor'].values[0]
    # print(tipo(saldo))

    popup_text = f"""
    <b>{pais}</b><br>
    📤 Exportado: {exp} MWh<br>
    📥 Importado: {imp} MWh<br>
    ⚖️ Saldo: {saldo} MWh<br>
    """
    if saldo < 0:
        color = 'green'
    else:
        color = 'red'
    

    folium.Marker(
        location=[lat, lon],
        popup=popup_text,
        icon=folium.Icon(color=color, icon="info-sign")
    ).add_to(spain_map)

folium.Choropleth(
    geo_data=geo_json_data,
    data=df_saldo[df_saldo['tipo'] == 'saldo'],
    columns=["frontera", "valor"],
    nan_fill_color="gray",
    nan_fill_opacity=0.4,
    key_on="feature.properties.ADMIN",
    fill_color="RdYlGn",  # Escala de color de folium
    fill_opacity=0.7,
    line_opacity=0.2,
).add_to(spain_map)

spain_map

NameError: name 'df_intercambio_total' is not defined

### Generación


In [79]:
df_generacion_filtrado_24 = df_generacion[df_generacion['año'] == 2024]
df_generacion_filtrado_22 = df_generacion[df_generacion['año'] == 2022]

In [80]:
df_concat = pd.concat([df_generacion_filtrado_24, df_generacion_filtrado_22], ignore_index=True)
df_concat

,titulo,tipo,composite,ultima_actualizacion,fecha_extraccion,categoria,zona,valor,porcentaje,fecha,año,mes,dia,dia_semana,ID
0,Hidráulica,Renovable,0,2025-01-28,2025-02-19,Generación por tecnología,nacional,58381.148,0.097303,2024-01-01,2024,1,1,lunes,2025021987185
1,Hidráulica,Renovable,0,2025-01-28,2025-02-19,Generación por tecnología,nacional,68004.167,0.092353,2024-01-02,2024,1,2,martes,2025021987186
2,Hidráulica,Renovable,0,2025-01-28,2025-02-19,Generación por tecnología,nacional,93862.430,0.126579,2024-01-03,2024,1,3,miércoles,2025021987187
3,Hidráulica,Renovable,0,2025-01-28,2025-02-19,Generación por tecnología,nacional,139605.013,0.211274,2024-01-04,2024,1,4,jueves,2025021987188
4,Hidráulica,Renovable,0,2025-01-28,2025-02-19,Generación por tecnología,nacional,83467.494,0.107495,2024-01-05,2024,1,5,viernes,2025021987189
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38407,Cogeneración,No renovable,0,2022-05-26,2025-02-19,Generación por tecnología,Región de Murcia,80701.361,0.057386,2022-08-01,2022,8,1,lunes,2025021965635
38408,Cogeneración,No renovable,0,2022-05-26,2025-02-19,Generación por tecnología,Región de Murcia,52572.930,0.049885,2022-09-01,2022,9,1,jueves,2025021965636
38409,Cogeneración,No renovable,0,2022-05-26,2025-02-19,Generación por tecnología,Región de Murcia,53683.588,0.054182,2022-10-01,2022,10,1,sábado,2025021965637
38410,Cogeneración,No renovable,0,2022-05-26,2025-02-19,Generación por tecnología,Región de Murcia,57330.527,0.082296,2022-11-01,2022,11,1,martes,2025021965638


In [95]:
type(df_concat['fecha'])

pandas.core.series.Series

In [113]:
l = [1, 1]
m = str(l)
# "".join(str(l))
m

'[1, 1]'

In [ ]:
# Filtrar solo los años de interés y la zona 'nacional'
df_filtered = df_concat[(df_concat['año'].isin([2022, 2024])) & (df_concat['zona'] == 'nacional')].copy()

# Agrupar para asegurarnos de que solo haya un valor por día y año (sumamos la generación)
df_filtered = df_filtered.groupby(['año', 'mes', 'dia'], as_index=False).agg({'valor': 'sum'})
df_filtered
# Crear la columna con fecha sin el año real
df_filtered['fecha_sin_year'] = pd.to_datetime(df_filtered[['mes', 'dia']].rename(columns={'mes': 'month', 'dia': 'day'}).assign(year=2000))
df_filtered.head(50)




In [ ]:
# Graficar los datos
fig = px.line(
    df_filtered,
    x='fecha_sin_year',
    y='valor',
    color='año',
    title="Generación de energía a lo largo del año (Zona Nacional)",
    labels={'fecha_sin_year': 'Fecha', 'valor': 'Generación (GWh)', 'año': 'Año'}
)

fig.update_xaxes(tickformat="%m-%b")

fig.show()

In [75]:
df_intercambios_2y = df_intercambios[(df_intercambios['año'] == 2022) | (df_intercambios['año'] == 2024)]
df_intercambios_2y = df_intercambios_2y[df_intercambios_2y['frontera'].isin(['Francia', 'Portugal', 'Marruecos', 'Andorra'])].reset_index(drop=True)
df_intercambios_2y = df_intercambios_2y.groupby(['mes', 'tipo', 'año']).sum('valor').reset_index()
df_intercambios_2y['valor'] = df_intercambios_2y['valor'].apply(lambda x: abs(x))
df_intercambios_2y

,mes,tipo,año,composite,valor,porcentaje,dia,ID
0,1,export,2022,0,2055906.984,85.225440,1984,251102717255834
1,1,export,2024,0,1900027.114,81.347475,1984,251102718720522
2,1,import,2022,0,875929.211,38.774560,1984,251102717301094
3,1,import,2024,0,1410979.301,42.652525,1984,251102718769750
4,2,export,2022,0,1782735.936,78.159809,1624,226802454298896
5,2,export,2024,0,2173778.045,88.107905,1740,234902543322678
6,2,import,2022,0,1314824.592,33.840191,1624,226802454339776
7,2,import,2024,0,1053157.782,27.892095,1740,234902543368730
8,3,export,2022,0,1839121.201,85.857046,1984,251102717263150
9,3,export,2024,0,2159654.659,94.874299,1984,251102718727962


In [76]:
media = df_intercambios_2y['valor'].mean()
mediana = df_intercambios_2y['valor'].median()
maximo = df_intercambios_2y['valor'].max()
minimo = df_intercambios_2y['valor'].min()

fig = px.line(df_intercambios_2y, 
        x="mes", 
        y="valor", 
        color='tipo',
        line_dash="año",  # Diferenciar los años con estilos de línea
        markers=True, 
        title="Evolución Diaria de Importaciones y Exportaciones (2022 vs 2024)",
        labels={"valor": "Demanda (MWh)", "fecha": "Fecha"},
        color_discrete_map={"export": "green", "import": "red"}  # Colores personalizados
        )

fig.add_annotation(
    x=0.5, y=0.95, text=f"Media: {media:.2f}", showarrow=False, font=dict(size=12), align="center", bgcolor="rgba(255,255,255,0.7)"
)
fig.add_annotation(
    x=0.5, y=0.90, text=f"Mediana: {mediana:.2f}", showarrow=False, font=dict(size=12), align="center", bgcolor="rgba(255,255,255,0.7)"
)
fig.add_annotation(
    x=0.5, y=0.85, text=f"Máximo: {maximo:.2f}", showarrow=False, font=dict(size=12), align="center", bgcolor="rgba(255,255,255,0.7)"
)
fig.add_annotation(
    x=0.5, y=0.80, text=f"Mínimo: {minimo:.2f}", showarrow=False, font=dict(size=12), align="center", bgcolor="rgba(255,255,255,0.7)"
)
        
                 

In [86]:
df_generacion

,titulo,tipo,composite,ultima_actualizacion,fecha_extraccion,categoria,zona,valor,porcentaje,fecha,año,mes,dia,dia_semana,ID
0,Hidráulica,Renovable,0,2021-01-12,2025-02-19,Generación por tecnología,nacional,151481.978,0.258134,2020-01-01,2020,1,1,miércoles,202502190
1,Hidráulica,Renovable,0,2021-01-12,2025-02-19,Generación por tecnología,nacional,159171.675,0.229885,2020-01-02,2020,1,2,jueves,202502191
2,Hidráulica,Renovable,0,2021-01-12,2025-02-19,Generación por tecnología,nacional,151996.921,0.226005,2020-01-03,2020,1,3,viernes,202502192
3,Hidráulica,Renovable,0,2021-01-12,2025-02-19,Generación por tecnología,nacional,139873.002,0.209615,2020-01-04,2020,1,4,sábado,202502193
4,Hidráulica,Renovable,0,2021-01-12,2025-02-19,Generación por tecnología,nacional,126801.179,0.210290,2020-01-05,2020,1,5,domingo,202502194
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99018,Cogeneración,No renovable,0,2025-02-05,2025-02-19,Generación por tecnología,Región de Murcia,118232.835,0.146870,2024-08-01,2024,8,1,jueves,20250219111167
99019,Cogeneración,No renovable,0,2025-02-05,2025-02-19,Generación por tecnología,Región de Murcia,97220.279,0.148079,2024-09-01,2024,9,1,domingo,20250219111168
99020,Cogeneración,No renovable,0,2025-02-05,2025-02-19,Generación por tecnología,Región de Murcia,72662.688,0.114139,2024-10-01,2024,10,1,martes,20250219111169
99021,Cogeneración,No renovable,0,2025-02-05,2025-02-19,Generación por tecnología,Región de Murcia,115075.762,0.171013,2024-11-01,2024,11,1,viernes,20250219111170


In [74]:
year1 = 2022
year2 = 2024

In [89]:
df_generacion_f = df_generacion[df_generacion['zona'] == 'nacional']
df_generacion_f = df_generacion_f[(df_generacion['año'] == year1) | (df_generacion['año'] == year2)]
# df_generacion_f['mes_dia'] = df_generacion_f['fecha'].apply(lambda x: x[5:11])
df_generacion_f = df_generacion_f.groupby(['mes', 'año']).sum('valor').reset_index()
df_generacion_f



C:\Users\armej\AppData\Local\Temp\ipykernel_20604\691024796.py:2: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



,mes,año,composite,valor,porcentaje,dia,ID
0,1,2022,0,2.408670e+07,31.0,7440,941635205078418
1,1,2024,0,2.312715e+07,31.0,7440,941635225403072
2,2,2022,0,2.083565e+07,28.0,6090,850509217502574
3,2,2024,0,2.178124e+07,29.0,6525,880884565712698
4,3,2022,0,2.249587e+07,31.0,7440,941635205105853
5,3,2024,0,2.203221e+07,31.0,7458,943660247419405
6,4,2022,0,2.143581e+07,30.0,6975,911259875922615
7,4,2024,0,2.072055e+07,30.0,6975,911259895592085
8,5,2022,0,2.175847e+07,31.0,7440,941635205134218
9,5,2024,0,2.085135e+07,31.0,7440,941635225459337


In [90]:
fig = px.line(df_generacion_f, 
        x="mes", 
        y="valor", 
        color='año',
        markers=True, 
        title="Evolución de la generación (2022 vs 2024)",
        labels={"valor": "Demanda (MWh)", "fecha": "Fecha"},
        color_discrete_map={"export": "green", "import": "red"}  # Colores personalizados
        )

fig.show()

In [91]:
df_balance

,titulo,tipo,composite,ultima_actualizacion,fecha_extraccion,categoria,zona,valor,porcentaje,fecha,año,mes,dia,dia_semana,ID
0,Hidráulica,Renovable,0,2021-01-12,2025-02-19,Balance de energía eléctrica,nacional,151481.978,0.708451,2020-01-01,2020,1,1,miércoles,202502190
1,Hidráulica,Renovable,0,2021-01-12,2025-02-19,Balance de energía eléctrica,nacional,159171.675,0.589429,2020-01-02,2020,1,2,jueves,202502191
2,Hidráulica,Renovable,0,2021-01-12,2025-02-19,Balance de energía eléctrica,nacional,151996.921,0.572911,2020-01-03,2020,1,3,viernes,202502192
3,Hidráulica,Renovable,0,2021-01-12,2025-02-19,Balance de energía eléctrica,nacional,139873.002,0.463375,2020-01-04,2020,1,4,sábado,202502193
4,Hidráulica,Renovable,0,2021-01-12,2025-02-19,Balance de energía eléctrica,nacional,126801.179,0.544133,2020-01-05,2020,1,5,domingo,202502194
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119549,Saldo de intercambios,Saldo de intercambios,0,2025-02-10,2025-02-19,Balance de energía eléctrica,Región de Murcia,82415.418,0.092869,2024-08-01,2024,8,1,jueves,20250219159036
119550,Saldo de intercambios,Saldo de intercambios,0,2025-02-10,2025-02-19,Balance de energía eléctrica,Región de Murcia,101140.254,0.133486,2024-09-01,2024,9,1,domingo,20250219159037
119551,Saldo de intercambios,Saldo de intercambios,0,2025-02-10,2025-02-19,Balance de energía eléctrica,Región de Murcia,101984.722,0.138078,2024-10-01,2024,10,1,martes,20250219159038
119552,Saldo de intercambios,Saldo de intercambios,0,2025-02-10,2025-02-19,Balance de energía eléctrica,Región de Murcia,49044.476,0.067933,2024-11-01,2024,11,1,viernes,20250219159039


In [94]:
df_demanda

,titulo,composite,ultima_actualizacion,fecha_extraccion,categoria,zona,valor,porcentaje,fecha,año,mes,dia,dia_semana,ID
0,Demanda,0,2021-01-12,2025-02-19,Evolución de la demanda,nacional,574776.720000,1.0,2020-01-01,2020,1,1,miércoles,202502190
1,Demanda,0,2021-01-12,2025-02-19,Evolución de la demanda,nacional,723522.375000,1.0,2020-01-02,2020,1,2,jueves,202502191
2,Demanda,0,2021-01-12,2025-02-19,Evolución de la demanda,nacional,754645.342000,1.0,2020-01-03,2020,1,3,viernes,202502192
3,Demanda,0,2021-01-12,2025-02-19,Evolución de la demanda,nacional,702390.878000,1.0,2020-01-04,2020,1,4,sábado,202502193
4,Demanda,0,2021-01-12,2025-02-19,Evolución de la demanda,nacional,651967.486000,1.0,2020-01-05,2020,1,5,domingo,202502194
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18928,Variación mensual corregida,1,2025-01-22,2025-02-19,Índice de Red Eléctrica General (IRE),nacional,5.186531,1.0,2024-08-01,2024,8,1,jueves,2025021926435
18929,Variación mensual corregida,1,2025-01-22,2025-02-19,Índice de Red Eléctrica General (IRE),nacional,4.756270,1.0,2024-09-01,2024,9,1,domingo,2025021926436
18930,Variación mensual corregida,1,2025-01-22,2025-02-19,Índice de Red Eléctrica General (IRE),nacional,5.356519,1.0,2024-10-01,2024,10,1,martes,2025021926437
18931,Variación mensual corregida,1,2025-01-22,2025-02-19,Índice de Red Eléctrica General (IRE),nacional,-3.795579,1.0,2024-11-01,2024,11,1,viernes,2025021926438
